In [1]:
##### VacationPy code - create heatmap and pin markers for hotels in cities meeting the criteria
# Dependencies

from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd

import requests
import json

import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

from pprint import pprint

In [2]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [3]:
# read in the csv created in the WeatherPy section
city_data = pd.read_csv("../WeatherPy/Output/City_DataFrame_CSV_Output.csv",dtype="object", encoding="utf-8")

In [4]:
# Fix data types
city_data[["City Latitude", "City Longitude",'Temperature (F)','Humidity %','Cloud Coverage %','Wind Speed (mph)']] = city_data[["City Latitude", "City Longitude",'Temperature (F)','Humidity %','Cloud Coverage %','Wind Speed (mph)']].astype(float)

In [5]:
# Check City_Data loaded correctly
city_data.tail(2)

,City Name,Temperature (F),Humidity %,Cloud Coverage %,Wind Speed (mph),City Latitude,City Longitude,City Country,City Local Date_Time,Random_Latitude,Random_Longitude
539,Buluang,75.34,87.0,100.0,4.43,13.3068,123.3425,PH,"Fri, 12 Mar 2021 01:16:12",13.43,116.32
540,Ixtapa,78.80,53.0,1.0,5.35,20.7000,-105.2000,MX,"Thu, 11 Mar 2021 11:16:13",9.68,-105.08


In [6]:
# set up data for heat map on humidity
locations = city_data[["City Latitude", "City Longitude"]]
humidity  = city_data["Humidity %"]

In [11]:
# Display heat map for humidity
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,opacity=.7, 
                                 max_intensity=100,point_radius = 2.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# restrict the number of cities so we have < 10
ideal_cities = city_data[((city_data["Temperature (F)"]) >= 72) & ((city_data["Temperature (F)"]) <= 79) & ((city_data["Wind Speed (mph)"]) >= 5) & ((city_data["Wind Speed (mph)"]) <= 10)& ((city_data["Cloud Coverage %"]) >= 0) & ((city_data["Cloud Coverage %"]) <= 10)]

In [13]:
# Check our count < 10 and all values populated
ideal_cities.count()

City Name               7
Temperature (F)         7
Humidity %              7
Cloud Coverage %        7
Wind Speed (mph)        7
City Latitude           7
City Longitude          7
City Country            7
City Local Date_Time    7
Random_Latitude         7
Random_Longitude        7
dtype: int64

In [14]:
# Part 1 set up to look up geocoordinates and add hotel info
ideal_cities['Hotel_Name'] = ""
ideal_cities['Hotel_Address'] = ""
ideal_cities['Hotel_Lat'] = ""
ideal_cities['Hotel_Lng'] = ""
ideal_cities.tail(2)

C:\Users\edwar\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\edwar\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\edwar\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,City Name,Temperature (F),Humidity %,Cloud Coverage %,Wind Speed (mph),City Latitude,City Longitude,City Country,City Local Date_Time,Random_Latitude,Random_Longitude,Hotel_Name,Hotel_Address,Hotel_Lat,Hotel_Lng
492,Lumphat,76.41,59.0,0.0,9.78,13.4878,106.9816,KH,"Fri, 12 Mar 2021 00:15:57",13.94,106.95,,,,
540,Ixtapa,78.80,53.0,1.0,5.35,20.7000,-105.2000,MX,"Thu, 11 Mar 2021 11:16:13",9.68,-105.08,,,,


In [15]:
# Part 2 create the hotel label markers
# gather the info for the hotels (lodging) within 5000 meters of the city location
for index, row in ideal_cities.iterrows():
    # set up a parameters dictionary
    lat = row["City Latitude"]
    lng = row["City Longitude"]

    params = {
        "location": f"{lat},{lng}",
        "radius": "5000",
        "types": "lodging",
        "key": gkey
    }
    

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # print(f"{lat},{lng}")
# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
# trap any situations where no info returned
    try:
        ideal_cities.loc[index,'Hotel_Name'] = places_data["results"][0]["name"]
        ideal_cities.loc[index,'Hotel_Address'] = places_data["results"][0]["vicinity"]
        ideal_cities.loc[index,'Hotel_Lat'] = places_data['results'][0]['geometry']['location']['lat']
        ideal_cities.loc[index,'Hotel_Lng'] = places_data['results'][0]['geometry']['location']['lng']
     
    except (KeyError, IndexError):
        # if a bad return we do not want this city data, so drop it from the file
        print(f"For City: {ideal_cities.loc[index,'City Name']}, Missing field/result... deleting.")
        ideal_cities.drop(index, inplace=True)

C:\Users\edwar\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [16]:
# build a list of the hotel info in a dictionary format so we can use 3 values for the markers
hotel_info = ideal_cities[['Hotel_Name','City Name','City Country']]
hotel_list = hotel_info.to_dict('records')

# check the dictionary built correctly
hotel_list

[{'Hotel_Name': 'CHACARA BAILLY', 'City Name': 'Castro', 'City Country': 'CL'},
 {'Hotel_Name': 'YASH HOTEL', 'City Name': 'Rādhanpur', 'City Country': 'IN'},
 {'Hotel_Name': 'Orient Apartments',
  'City Name': 'Nizwá',
  'City Country': 'OM'},
 {'Hotel_Name': 'Macs', 'City Name': 'Russell', 'City Country': 'NZ'},
 {'Hotel_Name': 'Sur Plaza Hotel', 'City Name': 'Sur', 'City Country': 'OM'},
 {'Hotel_Name': 'Lumphat Homestay',
  'City Name': 'Lumphat',
  'City Country': 'KH'},
 {'Hotel_Name': 'Hotel Plaza Providencia',
  'City Name': 'Ixtapa',
  'City Country': 'MX'}]

In [17]:
# Create a marker_layer using the hotel list to fill the info box and dispaly the merged map
# create the marker locations
marker_locations = ideal_cities[['Hotel_Lat','Hotel_Lng']]
      

In [19]:
# Create a template and insert into the marker code so we can insert the values from the dictionary list
info_box_template = """
<dl>
<dt><strong>Name</strong></dt><dd>{Hotel_Name}</dd>
<dt><strong>City</dt></strong><dd>{City Name}</dd>
<dt><strong>Country</strong></dt><dd>{City Country}</dd>
</dl>
"""

# build the marker info from the template and dictionary list
hotel_info = [info_box_template.format(**hotel) for hotel in hotel_list]

# create the marker layer, add to the heatmap and display
marker_layer = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))